# Ejercicio 04: Evaluación de un Sistema de Recuperación de Información

El objetivo de este ejercicio es evaluar la efectividad de un sistema de recuperación de información utilizando métricas como *precisión*, *recall*, *F1-score*, *Mean Average Precision (MAP)* y *Normalized Discounted Cumulative Gain (nDCG)*.

Seguirás los siguientes pasos:

Descripción del Ejercicio

1. Proporcionar un Conjunto de Datos:
    * Corpus de Documentos: Utiliza el corpus del ejercicio anterior o un nuevo conjunto de documentos.
    * Consultas: Define un conjunto de consultas específicas.
    * Juicios de Relevancia: Proporciona una lista de qué documentos son relevantes para cada consulta.

2. Calcular Resultados de Búsqueda:
    * Obten los resultados ordenados de dos sistemas de recuperación para cada consulta.

3. Calcular las Métricas de Evaluación:
    * Calcular las siguientes métricas para cada sistema y consulta:
        * Precisión en el top-k (Prec@k)
        * Recall
        * F1-score
        * Mean Average Precision (MAP)
        * nDCG

4. Análisis y Comparación:
    * Comparar los resultados de los dos sistemas utilizando las métricas calculadas.
    * Discutir cuál sistema es más efectivo y por qué.

## Corpus de Docuemntos, consultas definidas

In [49]:
# Importar las librerías necesarias
import xml.etree.ElementTree as ET

# Definir las consultas
queries = {
    1: "Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios",
    2: "Actividades extracurriculares y bienestar emocional en el campus universitario",
    3: "Estrategias universitarias para reducir el estrés en estudiantes"
}

# Función para procesar el texto y extraer palabras clave
def process_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Reemplazar caracteres no alfanuméricos por espacios
    import re
    text = re.sub(r'[^a-záéíóúñü]+', ' ', text)
    # Tokenizar y eliminar palabras vacías si es necesario
    tokens = text.strip().split()
    return set(tokens)

# Paso 1: Leer y parsear el archivo XML
def parse_corpus(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    corpus = {}
    for doc in root.findall('document'):
        doc_id = int(doc.get('id'))
        title = doc.find('title').text
        keywords = doc.find('keywords').text
        author = doc.find('author').text
        date = doc.find('date').text
        # Procesar las palabras clave
        keyword_set = process_text(keywords)
        corpus[doc_id] = {
            'title': title,
            'keywords': keyword_set,
            'author': author,
            'date': date
        }
    return corpus

corpus = parse_corpus('../content/sample_data/03ranking_corpus.xml')
corpus

{1: {'title': 'El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.',
  'keywords': {'crónica',
   'médica',
   'salud',
   'tecnología',
   'telemedicina',
   'tratamiento'},
  'author': 'Dr. Juan Pérez',
  'date': '2023-01-15'},
 2: {'title': 'Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.',
  'keywords': {'académico',
   'estudiantes',
   'mental',
   'nutrición',
   'rendimiento',
   'salud'},
  'author': 'Dra. María López',
  'date': '2023-02-10'},
 3: {'title': 'Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.',
  'keywords': {'amistad',
   'bienestar',
   'campus',
   'estudiantil',
   'relaciones',
   'sociales'},
  'author': 'Miguel Rodríguez',
  'date': '2023-03-05'},
 4: {'title': 'El rol de las bibliotecas universitarias en el fomento de la investigación académica.',
  'keywords': {'academia',
   'bibliotecas',
   'investigación',
   'recursos

## Resultados de busqueda

In [50]:
query_keywords = {qid: process_text(query) for qid, query in queries.items()}
document_texts = [" ".join(doc['keywords']) for doc in corpus.values()]
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
# Vectorización de documentos y consultas usando TF-IDF
vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform(document_texts)
query_vectors = vectorizer.transform([" ".join(query) for query in query_keywords.values()])
# similitud de Jaccard
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union) if union else 0

# relevancia y similitud entre cada consulta y documento
relevance_scores = {}
for qid, query_vector in enumerate(query_vectors):
    cos_sim = cosine_similarity(doc_vectors, query_vector).flatten()
    jaccard_sim = [jaccard_similarity(corpus[doc_id]['keywords'], query_keywords[qid + 1]) for doc_id in corpus]
    relevance_scores[qid + 1] = {
        doc_id: (cos_sim[idx], jaccard_sim[idx])
        for idx, doc_id in enumerate(corpus)
    }


In [51]:
# Sistema 1: Ranking basado en Similitud de Coseno
ranked_results_cosine = {}
for qid, query_vector in enumerate(query_vectors):
    cos_sim = cosine_similarity(doc_vectors, query_vector).flatten()
    ranked_results_cosine[qid + 1] = sorted(
        [(doc_id, score) for doc_id, score in enumerate(cos_sim, start=1)],
        key=lambda x: x[1], reverse=True
    )

# Convertir ranking al formato solo IDs de documentos
ranked_results_cosine = {
    qid: [doc_id for doc_id, _ in ranking]
    for qid, ranking in ranked_results_cosine.items()
}

In [52]:
# Sistema 2: Ranking basado en Similitud de Jaccard
ranked_results_jaccard = {}
for qid, query_set in query_keywords.items():
    jaccard_sim = [
        (doc_id, jaccard_similarity(corpus[doc_id]['keywords'], query_set))
        for doc_id in corpus
    ]
    ranked_results_jaccard[qid] = sorted(jaccard_sim, key=lambda x: x[1], reverse=True)

# Convertir ranking al formato solo IDs de documentos
ranked_results_jaccard = {
    qid: [doc_id for doc_id, _ in ranking]
    for qid, ranking in ranked_results_jaccard.items()
}

## Juicios de relevancia obtenidos desde el ranking


In [61]:
# Umbral para considerar un documento relevante
THRESHOLD = 0.28
# Lista de relevancia estimada a partir del umbral de similitud
relevant_docs = {}

for qid, scores in relevance_scores.items():
    relevant_docs[qid] = [
        doc_id for doc_id, (cosine_score, jaccard_score) in scores.items()
        if cosine_score > THRESHOLD or jaccard_score > THRESHOLD
    ]

# Resultados estimados
for qid, docs in relevant_docs.items():
    print(f"Consulta {qid}: Documentos relevantes estimados {docs}")

Consulta 1: Documentos relevantes estimados [2, 7, 11, 13, 14, 29]
Consulta 2: Documentos relevantes estimados [5, 15, 23]
Consulta 3: Documentos relevantes estimados [24]


## Metricas de evaluación

In [62]:
# bibliotecas necesarias
import numpy as np
from sklearn.metrics import average_precision_score

# Función auxiliar para crear un vector de relevancia binario
def binary_relevance_vector(ranked_docs, relevant_docs):
    return [1 if doc_id in relevant_docs else 0 for doc_id in ranked_docs]

# Prec@k
def precision_at_k(ranked_docs, relevant_docs, k):
    relevance_vector = binary_relevance_vector(ranked_docs[:k], relevant_docs)
    return np.mean(relevance_vector) if len(relevance_vector) > 0 else 0

# Recall
def recall(ranked_docs, relevant_docs):
    relevance_vector = binary_relevance_vector(ranked_docs, relevant_docs)
    return sum(relevance_vector) / len(relevant_docs) if len(relevant_docs) > 0 else 0

# F1-Score
def f1_score_at_k(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Mean Average Precision (MAP)
def mean_average_precision(ranked_docs, relevant_docs):
    relevance_vector = binary_relevance_vector(ranked_docs, relevant_docs)
    return average_precision_score(relevance_vector, relevance_vector) if len(relevance_vector) > 0 else 0

# nDCG
def ndcg(ranked_docs, relevant_docs):
    relevance_vector = binary_relevance_vector(ranked_docs, relevant_docs)
    dcg = sum(rel / np.log2(idx + 2) for idx, rel in enumerate(relevance_vector))
    idcg = sum(1 / np.log2(idx + 2) for idx in range(min(len(relevant_docs), len(ranked_docs))))
    return dcg / idcg if idcg > 0 else 0


## Analisis y comparación

In [63]:
# Evaluación de ambos sistemas de recuperación
def evaluate_system(ranked_results, relevant_docs, k=5):
    metrics = {}
    for qid, ranked_docs in ranked_results.items():
        precision = precision_at_k(ranked_docs, relevant_docs[qid], k)
        rec = recall(ranked_docs, relevant_docs[qid])
        f1 = f1_score_at_k(precision, rec)
        map_score = mean_average_precision(ranked_docs, relevant_docs[qid])
        ndcg_score = ndcg(ranked_docs, relevant_docs[qid])

        # Guardamos los resultados en un diccionario
        metrics[qid] = {
            'Prec@k': precision,
            'Recall': rec,
            'F1-Score': f1,
            'MAP': map_score,
            'nDCG': ndcg_score
        }
    return metrics

# métricas para el sistema basado en Coseno
metrics_cosine = evaluate_system(ranked_results_cosine, relevant_docs)
# métricas para el sistema basado en Jaccard
metrics_jaccard = evaluate_system(ranked_results_jaccard, relevant_docs)

# Imprimir resultados
print("Métricas para el Sistema basado en Similitud de Coseno:")
for qid, metric in metrics_cosine.items():
    print(f"Consulta {qid}: {metric}")

print("\nMétricas para el Sistema basado en Similitud de Jaccard:")
for qid, metric in metrics_jaccard.items():
    print(f"Consulta {qid}: {metric}")


Métricas para el Sistema basado en Similitud de Coseno:
Consulta 1: {'Prec@k': 1.0, 'Recall': 1.0, 'F1-Score': 1.0, 'MAP': 1.0, 'nDCG': 1.0}
Consulta 2: {'Prec@k': 0.6, 'Recall': 1.0, 'F1-Score': 0.7499999999999999, 'MAP': 1.0, 'nDCG': 1.0}
Consulta 3: {'Prec@k': 0.2, 'Recall': 1.0, 'F1-Score': 0.33333333333333337, 'MAP': 1.0, 'nDCG': 1.0}

Métricas para el Sistema basado en Similitud de Jaccard:
Consulta 1: {'Prec@k': 1.0, 'Recall': 1.0, 'F1-Score': 1.0, 'MAP': 1.0, 'nDCG': 0.9930783166417602}
Consulta 2: {'Prec@k': 0.6, 'Recall': 1.0, 'F1-Score': 0.7499999999999999, 'MAP': 1.0, 'nDCG': 0.8854598815714874}
Consulta 3: {'Prec@k': 0.2, 'Recall': 1.0, 'F1-Score': 0.33333333333333337, 'MAP': 1.0, 'nDCG': 0.6309297535714575}
